In [1]:
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService # Service 임포트
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# --- 1단계: 모든 상품의 고유 ID만 수집하는 함수 ---
def get_all_product_ids():
    """
    안정화된 Chrome 옵션과 명시적인 드라이버 경로를 사용하여,
    '버거' 카테고리 페이지에 직접 접속하고 모든 상품 ID를 안전하게 수집합니다.
    """
    # 1-1. Chrome 안정화 옵션 설정
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 백그라운드 실행을 원할 경우 주석 해제
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36") # 일반 사용자처럼 보이게 함

    # 1-2. 현재 폴더의 chromedriver.exe를 사용하도록 경로 명시
    service = ChromeService(executable_path='./chromedriver.exe')

    # 1-3. 옵션과 서비스를 적용하여 드라이버 실행
    driver = webdriver.Chrome(service=service, options=chrome_options)

    burger_page_url = "https://www.lotteeatz.com/brand/ria?categoryId=C1001"
    driver.get(burger_page_url)
    driver.maximize_window()
    product_ids = []

    try:
        print("✅ 1단계: '버거' 카테고리 페이지에 직접 접속했습니다.")

        # 1-4. 팝업 제거 (안정성을 위해 유지)
        try:
            popup_close_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup")))
            popup_close_button.click()
            print("✅ 1단계: 프로모션 팝업을 닫았습니다.")
        except TimeoutException:
            print("ℹ️ 1단계: 프로모션 팝업이 없습니다.")

        # 1-5. 상품 목록의 내용물이 채워질 때까지 정교하게 대기
        print("⏳ 상품 목록의 최종 렌더링을 기다립니다...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.prod-list > li[onclick*="goBrandDetail"]'))
        )
        print("✅ 상품 목록 렌더링 완료!")
        time.sleep(1)

        # 1-6. BeautifulSoup으로 ID 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_list = soup.select('ul.prod-list > li')

        for item in product_list:
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                product_id = onclick_attr.split("'")[1]
                product_ids.append(product_id)

        if not product_ids:
             print("⚠️ 1단계: ID를 수집하지 못했습니다.")
        else:
            print(f"✅ 1단계: 총 {len(product_ids)}개의 상품 ID 수집을 완료했습니다.")

    except TimeoutException:
        print("❌ 1단계: 대기 시간 초과! 상품 목록을 찾는 데 실패했습니다. 사이트 구조를 다시 확인해주세요.")
    except Exception as e:
        print(f"❌ 1단계: ID 수집 중 예상치 못한 오류가 발생했습니다: {e}")
    finally:
        driver.quit()
        print("✅ 1단계: 브라우저를 종료합니다.")
        return product_ids

# --- 2단계: ID 리스트를 기반으로 상세 정보를 수집하는 함수 ---
def get_product_details(product_ids):
    if not product_ids:
        print("⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.")
        return []

    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

    service = ChromeService(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)

    all_products_data = []

    for i, pid in enumerate(product_ids):
        try:
            detail_url = f"https://www.lotteeatz.com/products/introductions/{pid}"
            driver.get(detail_url)

            detail_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info")))
            driver.execute_script("arguments[0].click();", detail_button)

            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']")))

            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            name = detail_soup.select_one('div.prod-tit').text.strip()

            nutrition_info, allergy_info, origin_info = {}, "", {}

            nutrition_div = detail_soup.find('div', class_='btext-tit', string='영양소 정보')
            if nutrition_div:
                nutrition_table = nutrition_div.find_next_sibling('div', class_='tbl-info-wrap')
                if nutrition_table:
                    for row in nutrition_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        nutrition_info[key] = value

            allergy_p = detail_soup.find('div', class_='btext-tit', string='알러지 정보')
            if allergy_p:
                allergy_info = allergy_p.find_next_sibling('p', class_='btext').text.strip()

            origin_div = detail_soup.find('div', class_='btext-tit', string='원산지 정보')
            if origin_div:
                origin_table = origin_div.find_next_sibling('div', class_='tbl-info-wrap')
                if origin_table:
                    for row in origin_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        origin_info[key] = value

            product_data = {"id": pid, "name": name, "nutrition": nutrition_info, "allergy": allergy_info, "origin": origin_info}
            all_products_data.append(product_data)
            print(f"({i+1}/{len(product_ids)}) ✅ '{name}' 정보 수집 성공")

        except Exception as e:
            print(f"({i+1}/{len(product_ids)}) ❌ ID '{pid}' 처리 중 오류 발생: {e}")

    driver.quit()
    print("✅ 2단계: 브라우저를 종료합니다.")
    return all_products_data

# --- 3단계: 수집된 데이터를 파일로 저장하는 함수 ---
def save_data(data):
    # JSON 저장
    with open("lotte_eatz_burgers_final.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers_final.json")

    # CSV 저장
    csv_data = []
    for item in data:
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()]) if item.get('nutrition') else ""
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()]) if item.get('origin') else ""

        csv_data.append([
            item.get('id', ''),
            item.get('name', ''),
            nutrition_str,
            item.get('allergy', ''),
            origin_str
        ])

    with open("lotte_eatz_burgers_final.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers_final.csv")

# --- 메인 실행 블록 ---
if __name__ == '__main__':
    ids = get_all_product_ids()

    if ids:
        details = get_product_details(ids)
        if details:
            save_data(details)
            print("\n🎉 모든 크롤링 작업이 성공적으로 완료되었습니다!")
        else:
            print("\n⚠️ 2단계: 상세 정보 수집에 실패했습니다.")
    else:
        print("\n최종적으로 수집된 데이터가 없습니다.")

✅ 1단계: '버거' 카테고리 페이지에 직접 접속했습니다.
ℹ️ 1단계: 프로모션 팝업이 없습니다.
⏳ 상품 목록의 최종 렌더링을 기다립니다...
❌ 1단계: 대기 시간 초과! 상품 목록을 찾는 데 실패했습니다. 사이트 구조를 다시 확인해주세요.
✅ 1단계: 브라우저를 종료합니다.

최종적으로 수집된 데이터가 없습니다.
